In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:

!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2q6mpagv
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2q6mpagv
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=3d88837a701fa564ad1922237eccbaf986e15f7cb8037eb67cbf0e96e9eaf99f
  Stored in directory: /tmp/pip-ephem-wheel-cache-hqe5i409/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
	%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [28]:
%%cu
#include <bits/stdc++.h>

/*# include <cuda.h>
  # include <fstream>
  # include <iostream>
  # include <sstream>
  # include <string>*/

using namespace std;

__global__ void matproduct(int *l, int *m, int *n, int row1, int col1, int row2, int col2){
    int x = blockIdx.x;   // id of col2
    int y = blockIdx.y;   // id of row1
    int z = blockIdx.z;   // id of num_matrices

    int i;
    n[(col2 * row1 * z) + (col2 * y) + x] = 0;

    for (i = 0; i < col1; i++){
        n[(col2 * row1 * z) + (col2 * y) + x] += l[(col1 * row1 * z) + (col1 * y) + i] * m[(col2 * row2 * z) + (col2 * i) + x];
    }
}

int main(){
    int num_matrices; //1000
    int row1; //25
    int col1; //25
    int row2; //25
    int col2; //25

    ifstream file("mat_mul.txt");
    if (!file.is_open()){
        cerr << "Unable to open file." << endl;
        return 1;
    }

    string line;
    while (getline(file, line)){
        istringstream iss(line);
        int k, m, n, p;

        if (iss >> k >> m >> n >> p){
            cout << "K: " << k << ", M: " << m << ", N: " << n << ", p: " << p << endl;
            num_matrices = k;
            row1 = m;
            col1 = n;
            row2 = n;
            col2 = p;
        }
        else{
            cerr << "Failed to k, m, n, p from the line." << endl;
        }
    }

    // host variables
    int a[row1 * col1 * num_matrices];
    int b[row2 * col2 * num_matrices];
    int c[row1 * col2 * num_matrices];

    // device variables
    int *d, *e, *f;
    int i, j;

    cudaEvent_t start, end;
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    // Initialize matrices a and b
    int element = 1;
    for (int k = 0; k < num_matrices; k++){
        for (i = 0; i < row1; i++){
            for (j = 0; j < col1; j++){
                a[(col1 * row1 * k) + (col1 * i) + j] = 1;
            }
        }

        element = 2;
        for (i = 0; i < row2; i++){
            for (j = 0; j < col2; j++){
                b[(col2 * row2 * k) + (col2 * i) + j] = 1;
            }
        }
    }

    // allocate memory on device
    cudaMalloc((void **)&d, row1 * col1 * num_matrices * sizeof(int));
    cudaMalloc((void **)&e, row2 * col2 * num_matrices * sizeof(int));
    cudaMalloc((void **)&f, row1 * col2 * num_matrices * sizeof(int));

    // copy input data to the device
    cudaMemcpy(d, a, row1 * col1 * num_matrices * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(e, b, row2 * col2 * num_matrices * sizeof(int), cudaMemcpyHostToDevice);

    // grid(x_dimension, y_dimension, z_dimension of the grid)
    dim3 grid(col2, row1, num_matrices);

    // launch kernel
    matproduct<<<grid, 1>>>(d, e, f, row1, col1, row2, col2);     // single thread per block
    // Wait for GPU to finish before exiting
    cudaDeviceSynchronize();

    // copy data from device to host
    cudaMemcpy(c, f, row1 * col2 * num_matrices * sizeof(int), cudaMemcpyDeviceToHost);

    printf("\nProduct of matrices:\n");
    for (int k = 0; k < 1; k++){
        printf("Matrix %d:\n", k + 1);
        for (i = 0; i < row1; i++){
            for (j = 0; j < col2; j++){
                printf("%d\t", c[(col2 * row1 * k) + (col2 * i) + j]);
            }
            printf("\n");
        }
        printf("\n");
    }

    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, end);
    printf("\nTime taken: %f\n ", milliseconds);

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}

K: 5, M: 5, N: 5, p: 5

Product of matrices:
Matrix 1:
5	5	5	5	5	
5	5	5	5	5	
5	5	5	5	5	
5	5	5	5	5	
5	5	5	5	5	


Time taken: 0.379008
 
